In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib as  mpl

In [2]:
mpl.rcParams[u'font.sans-serif'] = ['simhei']
mpl.rcParams['axes.unicode_minus'] = False

In [3]:
def readData(y,s,c):
    df = pd.read_csv("lvr_land/"+ y +"-s" + s + "/" + c + "_lvr_land_A.csv",encoding='utf-8',low_memory=False)
    return df

In [4]:
def combineDF(df1,df2):
    df = pd.concat([df1,df2],axis = 0)
    return df

In [5]:
def selectFarmland(df):
    
    df = df[["鄉鎮市區","土地移轉總面積(平方公尺)","交易標的","都市土地使用分區","非都市土地使用分區","非都市土地使用編定","總價(元)","單價(元/平方公尺)"]]
    df = df[(df.交易標的 == "土地")]  
    df_farmland = df[(((df.非都市土地使用分區 == '特定農業區')|(df.非都市土地使用分區 == "一般農業區"))&(df.非都市土地使用編定 =="農牧用地"))]
    #排除都市土地
    df_farmland = df_farmland[df_farmland["都市土地使用分區"].isnull()]
    return df_farmland

In [6]:
def fillNanprice(df_farmland):
    #modify zero or nan price
    df_ok = df_farmland[(df_farmland["單價(元/平方公尺)"].notnull())&(df_farmland["單價(元/平方公尺)"] != "0")]
    df = df_farmland[(df_farmland["單價(元/平方公尺)"].isnull())|(df_farmland["單價(元/平方公尺)"] == "0")]
    df = df[(df['土地移轉總面積(平方公尺)'] != "0")&(df['總價(元)'] != "0")]
    df["單價(元/平方公尺)"] = calculateprice(df['總價(元)'],df['土地移轉總面積(平方公尺)'])
    df_farmland = pd.concat([df_ok,df],axis = 0)
    return df_farmland

In [7]:
def grouptown(df,list_all_town):
    grouped=pd.to_numeric(df_farmland['單價(元/平方公尺)']).groupby(df_farmland['鄉鎮市區'])
    town_dict = {}
   
    for key,group in grouped:
        m = grouped.count()
        town_dict[key]= m[key] 
        if key not in list_all_town:
            list_all_town.append(key)
    return town_dict


In [8]:
def transcationNum(df):
    return df['鄉鎮市區'].count()

In [9]:
def calculateprice(total_price,area):
    return pd.to_numeric(total_price)/pd.to_numeric(area)

In [10]:
def totalcalculate(key): 
    value = sum(key.values())   
    return value

In [11]:
def averagenum(num):
    nsum = 0
    for i in num:
        nsum = nsum + i
    return nsum / len(num)

In [17]:
list_cityfilename = ['A','B','C','D','E','F','G','H','J','K','M','N','P','Q','T','U','V']
list_city = ['台北市','台中市','基隆市','台南市','高雄市','新北市','宜蘭縣','桃園縣','新竹縣','苗栗縣','南投縣','彰化縣','雲林縣','嘉義縣','屏東縣','花蓮縣','台東縣']
#list_cityfilename = ['H','D','N','B','T']
#list_city = ['桃園縣','台南市','彰化縣','台中市','屏東縣']
i=0
taiwan_transacation = {}#各縣市各年交易量
for c in list_cityfilename: #every city
    
    s = 4 # from 101-s4 to 107-s2
    town_tran_dist = {}
    鄉鎮逐年交易量 = {}
    list_all_town = []
    for y in range(101,108): #every year
        df = pd.DataFrame()
        while s <= 4:
            # read data
            df_s =  readData(str(y),str(s),c)
            
            if c=='Q':# combine嘉義縣市(Q,I) 
                df_s = combineDF(df_s,readData(str(y),str(s),'I'))
            if c=='J':# combine新竹縣市(J,O)   
                df_s = combineDF(df_s,readData(str(y),str(s),'O'))
                
            df = combineDF(df,df_s)

            if y == 107 and s == 2:
                break
            s = s+1
        s=1
        # select farmland
        df_farmland = selectFarmland(df)
        # fill nan price
        df_farmland = fillNanprice(df_farmland)
        #鄉鎮各年交易量
        鄉鎮逐年交易量[str(y)] = grouptown(df_farmland,list_all_town)
        #縣市各年交易量
        town_tran_dist[y] = transcationNum(df_farmland)
        
    taiwan_transacation[list_city[i]]=town_tran_dist
    
    #縣市各年交易量
    print("縣市各年交易量:")
    print(list_city[i])
    print("|交易量（件）",end='')
    for key in range(101,108):
        print("|"+"{:.0f}".format(town_tran_dist[key]),end='')
    print("|")
    
    #     #plot 縣市各年交易量
#     plt.plot(town_tran_dist.keys(),town_tran_dist.values(),marker='o',linestyle='solid',color="#b5cf6b")
#     #plt.xticks(range(len(town_tran_dist)),town_tran_dist.keys(),rotation=90) 
#     plt.xlabel("年")
#     plt.ylabel("交易量")
#     plt.title(list_city[i] + " "+"逐年交易量")
#     plt.grid()
#         #plt.legend(loc=2, bbox_to_anchor=(1.05,1.0),borderaxespad = 0.)
#         #plt.subplots_adjust(right=0.7)
#     plt.savefig("交易量-"+list_city[i])
#     plt.show()
    
    
    #各鄉鎮101-107總交易量
    
    鄉鎮={}
    鄉鎮歷年交易量 = {}
    各鄉鎮加總 = {}
    for t in list_all_town:
        鄉鎮 = {}
        for y in range(101,108):
            if t in 鄉鎮逐年交易量[str(y)].keys():
                鄉鎮[y] = 鄉鎮逐年交易量[str(y)][t]
        鄉鎮歷年交易量[t] = 鄉鎮
        各鄉鎮加總[t]=totalcalculate(鄉鎮歷年交易量[t])
   
    #各鄉鎮101-107各年交易量
    print("各鄉鎮101-107各年交易量:")
    各年交易量 ={}
    for y in range(101,108):
        sorted_transcation = sorted(鄉鎮逐年交易量[str(y)].items(),key=lambda s:s[1], reverse=True)
        各年交易量[str(y)] = sorted_transcation
    
    print("|排名\年",end='')
    for y in range(101,108):
        print("|"+str(y),end='')
    print("|")
    for y in range(101,108):
        print("|------",end='')
    print("|")
    for h in range(len(list_all_town)):
        print("|"+str(h+1)+"|",end='')
        for y in range(101,108):
            if h>=len(各年交易量[str(y)]):
                print("-|",end='')
            else:
                print(各年交易量[str(y)][h][0]+" / "+str(各年交易量[str(y)][h][1])+"|",end="") 
        print("")
    
    
    #鄉鎮總交易量排名
    print("各鄉鎮101-107總交易量:")
    sorted_transcation = sorted(各鄉鎮加總.items(),key=lambda s:s[1], reverse=True)
    print("|排名|交易量|")
    print("|------|------|")
    for s in range(len(list_all_town)):
        print("|"+str(s+1)+"|",end='')
        print(sorted_transcation[s][0]+"/"+str(sorted_transcation[s][1])+"|") 
    i = i+1



縣市各年交易量:
台北市
|交易量（件）|0|0|0|0|0|0|0|
各鄉鎮101-107各年交易量:
|排名\年|101|102|103|104|105|106|107|
|------|------|------|------|------|------|------|
各鄉鎮101-107總交易量:
|排名|交易量|
|------|------|
縣市各年交易量:
台中市
|交易量（件）|155|1103|1829|1202|696|807|291|
各鄉鎮101-107各年交易量:
|排名\年|101|102|103|104|105|106|107|
|------|------|------|------|------|------|------|
|1|霧峰區 / 26|霧峰區 / 212|大甲區 / 289|大甲區 / 227|后里區 / 92|大甲區 / 86|霧峰區 / 41|
|2|烏日區 / 24|后里區 / 183|霧峰區 / 203|后里區 / 127|大甲區 / 76|霧峰區 / 80|大里區 / 38|
|3|大甲區 / 20|烏日區 / 161|后里區 / 181|新社區 / 126|外埔區 / 74|后里區 / 74|大甲區 / 27|
|4|后里區 / 16|大里區 / 92|外埔區 / 175|霧峰區 / 121|豐原區 / 61|大里區 / 74|神岡區 / 24|
|5|新社區 / 16|大甲區 / 86|新社區 / 151|外埔區 / 117|新社區 / 59|烏日區 / 71|大安區 / 22|
|6|大里區 / 13|新社區 / 86|烏日區 / 144|大安區 / 87|霧峰區 / 59|大安區 / 64|外埔區 / 21|
|7|神岡區 / 9|外埔區 / 74|大里區 / 136|神岡區 / 72|大安區 / 48|新社區 / 55|后里區 / 20|
|8|外埔區 / 8|神岡區 / 44|大安區 / 113|烏日區 / 69|烏日區 / 41|外埔區 / 52|新社區 / 18|
|9|大雅區 / 6|大安區 / 35|神岡區 / 102|大里區 / 60|潭子區 / 29|潭子區 / 51|大肚區 / 14|
|10|太平區 / 3|大雅區 / 30|大雅區 / 69|東勢區 / 48|神岡區 / 29

縣市各年交易量:
桃園縣
|交易量（件）|419|4115|3584|1719|1309|1649|706|
各鄉鎮101-107各年交易量:
|排名\年|101|102|103|104|105|106|107|
|------|------|------|------|------|------|------|
|1|大園區 / 100|大園區 / 1170|新屋區 / 753|新屋區 / 325|新屋區 / 211|觀音區 / 507|觀音區 / 177|
|2|新屋區 / 99|新屋區 / 957|大園區 / 732|大園區 / 240|觀音區 / 208|新屋區 / 253|大園區 / 107|
|3|觀音區 / 50|觀音區 / 708|觀音區 / 529|觀音區 / 237|大園區 / 178|大園區 / 163|新屋區 / 102|
|4|中壢區 / 46|龍潭區 / 298|龍潭區 / 347|龍潭區 / 181|龍潭區 / 131|龍潭區 / 139|楊梅區 / 70|
|5|龍潭區 / 35|楊梅區 / 256|楊梅區 / 309|大溪區 / 165|楊梅區 / 129|楊梅區 / 116|大溪區 / 51|
|6|楊梅區 / 31|中壢區 / 175|中壢區 / 208|楊梅區 / 158|大溪區 / 99|八德區 / 107|龍潭區 / 46|
|7|蘆竹區 / 20|蘆竹區 / 171|平鎮區 / 208|中壢區 / 104|八德區 / 92|中壢區 / 94|八德區 / 42|
|8|八德區 / 19|大溪區 / 142|蘆竹區 / 186|平鎮區 / 104|蘆竹區 / 89|大溪區 / 90|蘆竹區 / 36|
|9|大溪區 / 10|八德區 / 128|大溪區 / 183|蘆竹區 / 95|中壢區 / 77|平鎮區 / 84|中壢區 / 35|
|10|平鎮區 / 5|平鎮區 / 79|八德區 / 111|八德區 / 79|平鎮區 / 68|蘆竹區 / 72|平鎮區 / 31|
|11|桃園區 / 3|桃園區 / 20|桃園區 / 9|復興區 / 20|復興區 / 17|復興區 / 15|復興區 / 5|
|12|復興區 / 1|復興區 / 11|復興區 / 8|桃園區 / 11|桃園區 / 10|桃園區 / 9|桃園區 / 4|


縣市各年交易量:
嘉義縣
|交易量（件）|174|1178|1398|1390|1192|1254|551|
各鄉鎮101-107各年交易量:
|排名\年|101|102|103|104|105|106|107|
|------|------|------|------|------|------|------|
|1|新港鄉 / 23|水上鄉 / 142|水上鄉 / 176|新港鄉 / 147|六腳鄉 / 152|六腳鄉 / 145|新港鄉 / 81|
|2|朴子市 / 18|太保市 / 115|民雄鄉 / 167|民雄鄉 / 135|新港鄉 / 129|新港鄉 / 144|民雄鄉 / 63|
|3|民雄鄉 / 17|民雄鄉 / 113|六腳鄉 / 136|六腳鄉 / 134|大林鎮 / 111|大林鎮 / 116|六腳鄉 / 61|
|4|水上鄉 / 17|朴子市 / 98|太保市 / 124|水上鄉 / 131|水上鄉 / 110|民雄鄉 / 113|大林鎮 / 49|
|5|六腳鄉 / 16|新港鄉 / 91|新港鄉 / 118|太保市 / 122|溪口鄉 / 105|水上鄉 / 103|溪口鄉 / 38|
|6|中埔鄉 / 14|鹿草鄉 / 91|義竹鄉 / 101|大林鎮 / 110|義竹鄉 / 92|朴子市 / 89|鹿草鄉 / 33|
|7|大林鎮 / 13|六腳鄉 / 85|大林鎮 / 96|朴子市 / 105|民雄鄉 / 91|溪口鄉 / 79|朴子市 / 29|
|8|太保市 / 13|大林鎮 / 77|朴子市 / 89|鹿草鄉 / 93|朴子市 / 80|義竹鄉 / 78|太保市 / 28|
|9|溪口鄉 / 9|中埔鄉 / 71|鹿草鄉 / 88|義竹鄉 / 87|鹿草鄉 / 77|太保市 / 75|水上鄉 / 28|
|10|義竹鄉 / 7|溪口鄉 / 71|溪口鄉 / 85|竹崎鄉 / 74|太保市 / 64|東石鄉 / 69|竹崎鄉 / 26|
|11|鹿草鄉 / 7|義竹鄉 / 61|東石鄉 / 51|溪口鄉 / 73|竹崎鄉 / 59|鹿草鄉 / 65|中埔鄉 / 25|
|12|竹崎鄉 / 6|竹崎鄉 / 58|布袋鎮 / 41|東石鄉 / 54|東石鄉 / 48|中埔鄉 / 63|東石鄉 / 25|
|13|布袋鎮 / 5|東

In [13]:
y_taiwan_transcation = {}
for y in range(101,108):
    年度交易量 = []
    for c in list_city:      
        年度交易量.append(taiwan_transacation[c][y])
    y_taiwan_transcation[y] = sum(年度交易量)

#平均交易量
print("平均交易量")
print(averagenum(y_taiwan_transcation.values()))

for i in range(101,108):
    print("|"+str(i)+"|",end='')
    print(str(y_taiwan_transcation[i])+"|") 


平均交易量
16100.142857142857
|101|2727|
|102|21909|
|103|27021|
|104|21191|
|105|14838|
|106|16872|
|107|8143|


In [20]:
#縣市各年交易量排名
print("縣市各年交易量排名:")
table_dict = {}
for y in range(101,108):
    sorted_transcation = sorted(taiwan_transacation.items(),key=lambda s:s[1][y], reverse=True)
    table_dict[y] = sorted_transcation
    
print("|排名\年",end='')
for y in range(101,108):
    print("|"+str(y),end='')
print("|")
for y in range(101,108):
    print("|------",end='')
print("|")
for i in range(len(list_city)):
    print("|"+str(i+1)+"|",end='')
    for y in range(101,108):
        print(table_dict[y][i][0]+"/"+str(table_dict[y][i][1][y])+"|",end="") 
    print("")

縣市各年交易量排名:
|排名\年|101|102|103|104|105|106|107|
|------|------|------|------|------|------|------|
|1|桃園縣/419|桃園縣/4115|桃園縣/3584|台南市/2711|雲林縣/2027|台南市/2614|雲林縣/1145|
|2|屏東縣/343|台南市/2570|台南市/3069|彰化縣/2679|台南市/2015|雲林縣/2279|台南市/1129|
|3|台南市/337|彰化縣/2168|彰化縣/2962|屏東縣/2407|彰化縣/1839|彰化縣/2153|屏東縣/954|
|4|彰化縣/303|屏東縣/2106|屏東縣/2613|雲林縣/2172|屏東縣/1766|屏東縣/2098|彰化縣/952|
|5|雲林縣/227|雲林縣/2027|宜蘭縣/2332|桃園縣/1719|桃園縣/1309|桃園縣/1649|南投縣/926|
|6|宜蘭縣/212|宜蘭縣/1395|雲林縣/2241|宜蘭縣/1422|嘉義縣/1192|嘉義縣/1254|桃園縣/706|
|7|嘉義縣/174|嘉義縣/1178|台中市/1829|嘉義縣/1390|高雄市/880|高雄市/1004|嘉義縣/551|
|8|台中市/155|台中市/1103|花蓮縣/1466|高雄市/1320|宜蘭縣/701|台中市/807|高雄市/461|
|9|花蓮縣/131|新竹縣/1044|高雄市/1433|台中市/1202|台中市/696|南投縣/667|台中市/291|
|10|南投縣/125|南投縣/984|嘉義縣/1398|花蓮縣/1022|南投縣/611|宜蘭縣/554|宜蘭縣/250|
|11|高雄市/98|高雄市/947|南投縣/1192|南投縣/1016|新竹縣/524|苗栗縣/518|新竹縣/207|
|12|新竹縣/86|花蓮縣/890|新竹縣/1139|新竹縣/796|花蓮縣/446|新竹縣/481|花蓮縣/203|
|13|苗栗縣/78|苗栗縣/841|苗栗縣/1008|苗栗縣/694|苗栗縣/436|花蓮縣/439|苗栗縣/201|
|14|台東縣/28|台東縣/297|台東縣/455|台東縣/374|台東縣/237|台東縣/214|台東縣/108|
|15|新北市/11|新北市

In [19]:
#各縣市101-107總交易量
print("各縣市101-107總交易量:")
total_transcation = {}
for c in list_city:
    total_transcation[c] = totalcalculate(taiwan_transacation[c])

table_dict = {}
sorted_transcation = sorted(total_transcation.items(),key=lambda s:s[1], reverse=True)
print("")

print("|排名\年|交易量（件）|")

print("|------|------|")
for i in range(len(list_city)):
    print("|"+str(i+1)+"|",end='')
    print(sorted_transcation[i][0]+"/"+str(sorted_transcation[i][1])+"|") 

各縣市101-107總交易量:

|排名\年|交易量（件）|
|------|------|
|1|台南市/14445|
|2|桃園縣/13501|
|3|彰化縣/13056|
|4|屏東縣/12287|
|5|雲林縣/12118|
|6|嘉義縣/7137|
|7|宜蘭縣/6866|
|8|高雄市/6143|
|9|台中市/6083|
|10|南投縣/5521|
|11|花蓮縣/4597|
|12|新竹縣/4277|
|13|苗栗縣/3776|
|14|台東縣/1713|
|15|新北市/1181|
|16|台北市/0|
|17|基隆市/0|
